In [ ]:
import tkinter as tk
import tkinter.ttk as ttk

import imutils as imutils
from tkcalendar import DateEntry
from PIL import Image, ImageTk
import os
import cv2
from tkinter import messagebox
import DB
from ultralytics import YOLO


def login():
    loginData = DB.db.collection('users').where('tc', '==', tc_entry.get()).where('password', '==', password_entry.get()).get()
    if (len(loginData) == 0):
        messagebox.showerror(title='Try Again!', message='T.C. Identity Number or Password Incorrect!')
    else:
        form1.destroy()
        form2 = tk.Tk()
        form2.title("Laboratory Result System")

        form2.attributes('-zoomed', True)
        form2.resizable(False, False)
        form2.configure(bg='#72B6AB')

        def newRecord():
            if (
                    patient_tc_entry.get() != '' and patient_name_entry.get() != '' and patient_surname_entry.get() != '' and dateofBirth_entry.get() != '' and x.get() != '' and barcode_entry != ''):
                patientData = DB.db.collection('patients').where('tc', '==', patient_tc_entry.get()).where('barcode',
                                                                                                           '==',
                                                                                                           barcode_entry.get()).get()
                if (len(patientData) == 0):
                    patientTcData = DB.db.collection('patients').where('tc', '==', patient_tc_entry.get()).get()
                    if (len(patientTcData) == 0):
                        patientData = {'tc': patient_tc_entry.get(), 'name': patient_name_entry.get(),
                                       'surname': patient_surname_entry.get(), 'dateofBirth': dateofBirth_entry.get(),
                                       'gender': x.get()}
                        patientTest = {'barcode': barcode_entry.get(), 'requestHour': requestHour_entry.get(),
                                            'appHour': appHour_entry.get()}
                        DB.db.collection('patients').document(patient_tc_entry.get()).set(patientData)
                        DB.db.collection('patients').document(patient_tc_entry.get()).collection('tests').document(
                            barcode_entry.get()).set(patientTest)
                        messagebox.showinfo(title='Successful!', message='Patient Record Successfully Created.')
                    else:
                        patientTest = {'barcode': barcode_entry.get(), 'requestHour': requestHour_entry.get(),
                                            'appHour': appHour_entry.get()}
                        DB.db.collection('patients').document(patient_tc_entry.get()).collection('tests').document(
                            barcode_entry.get()).set(patientTest)
                        messagebox.showinfo(title='Successful!',
                                            message='Patient Record Successfully Created.')
                else:
                    messagebox.showinfo(title='Try Again!',
                                        message=patient_tc_entry.get() + ' Patient ID Number ' + barcode_entry.get() + ' Analysis with Barcode Number is Available in the System!')
            else:
                messagebox.showerror(title='Try Again!',
                                     message='All Fields Must Be Filled! \nPlease fill in the blanks!')

        formMenu2 = tk.Menu(form2)
        file2 = tk.Menu(formMenu2, tearoff=0)
        file2.add_command(label='New Record', command=newRecord)
        file2.add_command(label='Deregister')
        file2.add_separator()
        file2.add_command(label='Search')
        file2.add_separator()
        file2.add_command(label='Exit', command=form2.destroy)
        formMenu2.add_cascade(label='File', menu=file2)
        form2.config(menu=formMenu2)

        edit2 = tk.Menu(formMenu2, tearoff=0)
        edit2.add_command(label='Undo')
        edit2.add_separator()
        edit2.add_command(label='Cut')
        edit2.add_command(label='Copy')
        edit2.add_command(label='Paste')
        edit2.add_command(label='Delete')
        formMenu2.add_cascade(label='Edit', menu=edit2)
        form2.config(menu=formMenu2)

        labelFrame1 = tk.LabelFrame(form2, text='Patient Information', font='Times 15 bold italic', bg='#72B6AB', fg='white',
                                    labelanchor='n')
        labelFrame1.place(x=25, y=0, width=205, height=380)



        patient_tc = tk.Label(labelFrame1, text='T.C. Identification Number:', fg='white', font='Times 10',
                              bg='#72B6AB')
        patient_tc.place(x=20, y=0)
        patient_name = tk.Label(labelFrame1, text='Name:', fg='white', font='Times 10', bg='#72B6AB')
        patient_name.place(x=20, y=30)
        patient_surname = tk.Label(labelFrame1, text='Surname:', fg='white', font='Times 10', bg='#72B6AB')
        patient_surname.place(x=20, y=60)
        dateofBirth = tk.Label(labelFrame1, text='Date of Birth:', fg='white', font='Times 10', bg='#72B6AB')
        dateofBirth.place(x=20, y=90)
        gender = tk.Label(labelFrame1, text='Gender:', fg='white', font='Times 10', bg='#72B6AB')
        gender.place(x=20, y=120)
        barcode = tk.Label(labelFrame1, text='Barcode No:', fg='white', font='Times 10', bg='#72B6AB')
        barcode.place(x=20, y=150)
        requestHour = tk.Label(labelFrame1, text='Request Time:', fg='white', font='Times 10', bg='#72B6AB')
        requestHour.place(x=20, y=180)
        appHour = tk.Label(labelFrame1, text='Application Time:', fg='white', font='Times 10', bg='#72B6AB')
        appHour.place(x=20, y=210)
        conclusionHour = tk.Label(labelFrame1, text='Result Time:', fg='white', font='Times 10', bg='#72B6AB')
        conclusionHour.place(x=20, y=240)
        expertApprovalHour = tk.Label(labelFrame1, text='Confirmation Time:', fg='white', font='Times 10',
                                      bg='#72B6AB')
        expertApprovalHour.place(x=20, y=270)

        patient_tc_entry = tk.Entry(labelFrame1)
        patient_tc_entry.place(x=20, y=15, width=160, height=17)
        patient_name_entry = tk.Entry(labelFrame1)
        patient_name_entry.place(x=20, y=45, width=160, height=17)
        patient_surname_entry = tk.Entry(labelFrame1)
        patient_surname_entry.place(x=20, y=75, width=160, height=17)
        dateofBirth_entry = DateEntry(labelFrame1, locale='en_US', date_pattern='dd/mm/y', year=1980, month=1, day=1)
        dateofBirth_entry.place(x=20, y=105, width=160, height=17)
        x = tk.StringVar()
        gender_female = tk.Radiobutton(labelFrame1, text='Female', font='Times 8', value='Kadın', bg='#72B6AB', fg='white', activeforeground='#72B6AB',
                                       variable=x)
        gender_female.place(x=35, y=135)
        gender_male = tk.Radiobutton(labelFrame1, text='Male', font='Times 8', value='Erkek', bg='#72B6AB', fg='white', activeforeground='#72B6AB', variable=x)  # variable=x
        gender_male.place(x=125, y=135)
        barcode_entry = tk.Entry(labelFrame1)
        barcode_entry.place(x=20, y=165, width=160, height=17)
        requestHour_entry = DateEntry(labelFrame1, locale='en_US', date_pattern='dd/mm/y', year=2022, month=1, day=1)
        requestHour_entry.place(x=20, y=195, width=160, height=17)
        appHour_entry = DateEntry(labelFrame1, locale='en_US', date_pattern='dd/mm/y', year=2022, month=1, day=1)
        appHour_entry.place(x=20, y=225, width=160, height=17)
        conclusionHour_entry = DateEntry(labelFrame1, locale='en_US', date_pattern='dd/mm/y', year=2022, month=1, day=1)
        conclusionHour_entry.place(x=20, y=255, width=160, height=17)
        expertApprovalHour_entry = DateEntry(labelFrame1, locale='en_US', date_pattern='dd/mm/y', year=2022, month=1,
                                             day=1)
        expertApprovalHour_entry.place(x=20, y=285, width=160, height=17)

        labelFrame2 = tk.LabelFrame(form2, text='Microscope Image', font='Times 15 bold italic', bg='#72B6AB', fg='white',
                                    labelanchor='n')
        labelFrame2.place(x=267, y=0, width=295, height=180)
        labelFrame3 = tk.LabelFrame(form2, text='Processed Image', font='Times 15 bold italic', bg='#72B6AB', fg='white',
                                    labelanchor='n')
        labelFrame3.place(x=267, y=190, width=295, height=180)
        labelFrame4 = tk.LabelFrame(form2, text='Report', font='Times 15 bold italic', bg='#72B6AB',fg='white', labelanchor='n')
        labelFrame4.place(x=600, y=30, width=175, height=320)

        def showImage():


             kamera = cv2.VideoCapture(0)
             kamera.set(cv2.CAP_PROP_FRAME_WIDTH, 400)
             kamera.set(cv2.CAP_PROP_FRAME_HEIGHT, 400)
             while kamera.isOpened():
                 ret, frame = kamera.read()
                 frame = imutils.resize(frame, width=270, height=150)
                 if not ret:
                     print("Kamera okunamadı...")
                     break
                 cv2.imshow("Kamera", frame)
                 tus = cv2.waitKey(1)
                 if tus == ord('q'):
                     break
                 elif tus == ord('s'):
                     cv2.imwrite("/home/ta/Desktop/kapilleroskopi/test_images/" + barcode_entry.get() + ".jpg", frame)
                     newImg = ImageTk.PhotoImage(Image.open(
                              "/home/ta/Desktop/kapilleroskopi/test_images/" + barcode_entry.get() + ".jpg"))
                     showImageLabel = tk.Label(labelFrame2, image=newImg)  # , command=openImage
                     showImageLabel.image = newImg
                     showImageLabel.place(x=10, y=0, width=270, height=150)
                     break
             kamera.release()
             cv2.destroyAllWindows()



        showImageButton = tk.Button(labelFrame1, text='Microscope Image', font='Times 9 bold', border=0, bg='white', fg='#72B6AB', activebackground='#72B6AB', activeforeground='white', command=showImage)
        showImageButton.place(x=20, y=310, width=100, height=20)

        def scan():
            file = "/home/ta/Desktop/kapilleroskopi/test_images/" + barcode_entry.get() + ".jpg"
            if os.path.exists(file):
                img = cv2.imread(file)
                resize = cv2.resize(img, (640, 640))
                cv2.imwrite("/home/ta/Desktop/kapilleroskopi/resizeImg/" + barcode_entry.get() + ".jpg",
                            resize)
                source_path = "/usr/local/lib/python3.9/dist-packages/yolov5/resizeImg/" + barcode_entry.get() + ".jpg"
                imageName = barcode_entry.get()
                model=YOLO('yolov8s-seg.pt')
                model=YOLO('/home/ta/Desktop/kapilleroskopi/best.pt')
                model.predict("/home/ta/Desktop/kapilleroskopi/resizeImg/" + barcode_entry.get() + ".jpg", save=True, save_txt=True, imgsz=640, conf=0.2)
            else:
                messagebox.showerror(title='Failed', message=barcode_entry.get() + ' There Is No Patient With Barcode Number!')

        scanButton = tk.Button(labelFrame1, text='Scan', font='Times 9 bold', border=0, bg='white', fg='#72B6AB', activebackground='#72B6AB', activeforeground='white', command=scan)
        scanButton.place(x=125, y=310, width=55, height=20)

        def scannedImage():
            file = "/home/ta/Desktop/kapilleroskopi/runs/segment/predict/" + barcode_entry.get() + '.jpg'
            if os.path.exists(file):
                img = cv2.imread(file)
                resize = cv2.resize(img, (270, 150))
                cv2.imwrite("/home/ta/Desktop/kapilleroskopi/resizeImg/" + barcode_entry.get() + "result.jpg",
                            resize)

                newImg = ImageTk.PhotoImage(Image.open(
                    "/home/ta/Desktop/kapilleroskopi/resizeImg/" + barcode_entry.get() + "result.jpg"))


#
                scannedImageLabel = tk.Label(labelFrame3, image=newImg)
                scannedImageLabel.image = newImg
                scannedImageLabel.place(x=10, y=0, width=270, height=150)

            else:
                messagebox.showerror(title='Please Wait!', message='Scan In Progress!')

        scannedImageButton = tk.Button(labelFrame1, text='Processed Image', font='Times 9 bold', border=0, bg='white', fg='#72B6AB', activebackground='#72B6AB', activeforeground='white', command=scannedImage)
        scannedImageButton.place(x=20, y=335, width=100, height=20)

        def results():
            labelFrame4 = tk.LabelFrame(form2, text='Report', font='Times 15 bold italic', bg='#72B6AB',fg='white', labelanchor='n')
            labelFrame4.place(x=600, y=30, width=175, height=320)

            sLabel = tk.Label(labelFrame4,
                                  text="\n\n\nCapillaroscopy Results of Patient with\n " + barcode_entry.get() + " Barcode Number: ",
                                  fg='white', font='Times 7 bold', bg='#72B6AB')
            sLabel.pack(pady=5)
            file = "/home/ta/Desktop/kapilleroskopi/runs/segment/predict/labels/" + barcode_entry.get() + '.txt'
            if os.path.exists(file):
                f = open(file, 'r')
                s = str(f.read())
                ss = s.rsplit('\n')
                number = str(len(ss)-1)
                sLabel = tk.Label(labelFrame4, text='Number of Capillar Veins: ' + number, fg='white', font='Times 7 bold', bg='#72B6AB')
                sLabel.pack(pady=5)



            else:
                messagebox.showerror(title='Please Wait!', message='Scan In Progress!')

        resultReportButton = tk.Button(labelFrame1, text='Report', font='Times 9 bold', border=0, bg='white', fg='#72B6AB', activebackground='#72B6AB', activeforeground='white', command=results)
        resultReportButton.place(x=125, y=335, width=55, height=20)


def signin():
    for widget in frame2.winfo_children():
        widget.destroy()

    def userRegistration():
        if (
                signin_tc_entry.get() != '' and signin_name_entry.get() != '' and signin_surname_entry.get() != '' and signin_mail_entry.get() != '' and signin_password_entry.get() != '' and signin_passwordAgain_entry.get()):
            if (signin_password_entry.get() == signin_passwordAgain_entry.get()):
                userData = {'tc': signin_tc_entry.get(), 'name': signin_name_entry.get(),
                            'surname': signin_surname_entry.get(), 'mail': signin_mail_entry.get(), 'password': signin_password_entry.get()}
                DB.db.collection('users').add(userData)
                messagebox.showinfo(title='Successful!', message='Your Registration Has Been Successfully Created.')
            else:
                messagebox.showerror(title='Try Again!', message='Please Make Sure You Enter The Same Password!')
        else:
            messagebox.showerror(title='Try Again!', message='All Fields Must Be Filled! \nPlease fill in the blanks!')



    title = tk.Label(frame2, text='Registration Form', fg='#72B6AB', font='Times 20 bold', bg='white')
    title.place(x=40, y=10)

    signin_tc = tk.Label(frame2, text='T.C. Identification Number:', fg='gray', font='Times 12', bg='white')
    signin_tc.place(x=40, y=45)
    signin_name = tk.Label(frame2, text='Name:', fg='gray', font='Times 12', bg='white')
    signin_name.place(x=40, y=90)
    signin_surname = tk.Label(frame2, text='Surname:', fg='gray', font='Times 12', bg='white')
    signin_surname.place(x=40, y=135)
    signin_mail = tk.Label(frame2, text='E-mail:', fg='gray', font='Times 12', bg='white')
    signin_mail.place(x=40, y=180)
    signin_password = tk.Label(frame2, text='Password:', fg='gray', font='Times 12', bg='white')
    signin_password.place(x=40, y=225)
    signin_passwordAgain = tk.Label(frame2, text='Repeat Password:', fg='gray', font='Times 12', bg='white')
    signin_passwordAgain.place(x=40, y=270)

    signin_tc_entry = tk.Entry(frame2, bg='white')
    signin_tc_entry.place(x=40, y=65, width=220, height=25)
    signin_name_entry = tk.Entry(frame2, bg='white')
    signin_name_entry.place(x=40, y=110, width=220, height=25)
    signin_surname_entry = tk.Entry(frame2, bg='white')
    signin_surname_entry.place(x=40, y=155, width=220, height=25)
    signin_mail_entry = tk.Entry(frame2, bg='white')
    signin_mail_entry.place(x=40, y=200, width=220, height=25)
    signin_passwordAgain_entry = tk.Entry(frame2, bg='white', show='*')
    signin_passwordAgain_entry.place(x=40, y=245, width=220, height=25)
    signin_password_entry = tk.Entry(frame2, bg='white', show='*')
    signin_password_entry.place(x=40, y=290, width=220, height=25)
    signin_Button = tk.Button(frame2, text='Register', font='Times 13 bold', bg='#72B6AB', fg='white', border=0, activeforeground='#72B6AB', activebackground='white', command=userRegistration)
    signin_Button.place(x=40, y=325, width=220)


form1 = tk.Tk()
form1.title("Laboratory Result System")

form1.attributes('-zoomed', True)
form1.resizable(False, False)

formMenu1 = tk.Menu(form1)
file1 = tk.Menu(formMenu1, tearoff=0)
file1.add_command(label='Admin Login')
file1.add_separator()
file1.add_command(label='Exit', command=form1.destroy)
formMenu1.add_cascade(label='File', menu=file1)
form1.config(menu=formMenu1)

edit1 = tk.Menu(formMenu1, tearoff=0)
edit1.add_command(label='Undo')
edit1.add_separator()
edit1.add_command(label='Cut')
edit1.add_command(label='Copy')
edit1.add_command(label='Paste')
edit1.add_command(label='Delete')
formMenu1.add_cascade(label='Edit', menu=edit1)
form1.config(menu=formMenu1)

settings1 = tk.Menu(formMenu1, tearoff=0)
language_menu = tk.Menu(settings1, tearoff=0)
language_menu.add_command(label='English')
language_menu.add_command(label='Turkish')
settings1.add_cascade(label='Languages', menu=language_menu)
formMenu1.add_cascade(label='Settings', menu=settings1)
form1.config(menu=formMenu1)

bg1 = ImageTk.PhotoImage(Image.open("/home/ta/Desktop/kapilleroskopi/GUI_images/bg11111.jpg"))
frame1 = tk.Frame(form1, bg='white')
frame1.place(width=800, height=480, anchor=tk.CENTER, relx=0.5, rely=0.5)
bg1label = tk.Label(frame1, image=bg1)
bg1label.image = bg1
bg1label.place(x=-1, y=40)


frame2 = tk.Frame(bg1label, bg='white')
frame2.place(x=25, y=20, width=300, height=360)
icon1 = ImageTk.PhotoImage(Image.open("/home/ta/Desktop/kapilleroskopi/GUI_images/icon1.png"))
bg1label = tk.Label(frame2, image=icon1, border=0)
bg1label.image = icon1
bg1label.place(x=100, y=20)

title = tk.Label(frame2, text='User Login', fg='#72B6AB', font='Times 20 bold', bg='white')
title.place(x=90, y=120)

tc = tk.Label(frame2, text='T.C. Identification Number:', fg='gray', font='Times 13', bg='white')
tc.place(x=40, y=170)
password = tk.Label(frame2, text='Password:', fg='gray', font='Times 13', bg='white')
password.place(x=40, y=220)

tc_entry = tk.Entry(frame2, bg='white')
tc_entry.place(x=40, y=195, width=220, height=25)
password_entry = tk.Entry(frame2,bg='white', show='*')
password_entry.place(x=40, y=245, width=220, height=25)

loginButton = tk.Button(frame2, text='Log In', font='Times 13 bold', bg='#72B6AB', fg='white', border=0, activeforeground='#72B6AB', activebackground='white', command=login) #, command=login
loginButton.place(x=40, y=280, width=220)

login_signinButton = tk.Button(frame2, text='Register', font='Times 13 bold underline', bg='white', fg='#72B6AB', border=0, activeforeground='#72B6AB', activebackground='white', command=signin) #, command=signin
login_signinButton.place(x=105, y=320)

form1.mainloop()